In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
import duckdb
import yaml
import numpy as np

Instantiating the webdriver

In [19]:
driver = webdriver.Edge()

# **Defining functions for webscrapping**

In [34]:
class DelegateRoster:
    
    def __init__(self):
        
        self.base_url = 'https://ocimpact.com/delegate-roster/'
    
    
    def get_links(self):
        '''
        This function extracts all links of the delegates.
        Args:
            None
        Returns:
            list of all the delegates' links
        '''
        #open website
        driver.get(self.base_url)
        WebDriverWait(driver, 100).until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[1]/div/div/div[2]/div/div/div/div[6]/div/div/div/div/div/div/p/iframe')))
        
        #switching frames
        frame = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/div/div/div/div[6]/div/div/div/div/div/div/p/iframe')
        driver.switch_to.frame(frame)
        
        #scroll down to the bottom of the page & keep on scrolling until we reached the end
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            sleep(2)
            
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
                
            last_height = new_height
        
        #delegates links
        delegates = driver.find_elements(By.XPATH, '//div[@class = "container__Grid-cmp__sc-1smp56b-1 dWWagW"]/a')
        delegates_links = []
        for i in range(0, len(delegates)):
            delegates_links.append(delegates[i].get_attribute('href'))
            
        return delegates_links
        
        
    
    def create_dataframe(self, delegates_links, limit):
        '''
        This function open those individual delegates links & extract the relevant information that is needed.
        Args:
            delegates_links: list of all the links
            limit: number of delegates that we want to extract info. 
                   Its value should be the length of the list
                   but in case if we want to restrict to first few delegates then we can provide that number.
        Returns:
            pandas dataframe containing all the info of the delegates
        '''
        
        #initialize a dictionary to store values
        d = {}
        
        
        for i in range(0, len(delegates_links)):
            
            if i == limit:
                break
            d[i] = {}  
            driver.get(delegates_links[i])
            WebDriverWait(driver, 1000).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[2]')))
            sleep(10)
            
            #name
            try:
                name = driver.find_element(By.XPATH, '//h2[@class = "style__Name-cmp__sc-1s7e137-1 jhjTCw"]').text
               # name = driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[2]/div/div[2]/h2').text
            except NoSuchElementException:
                name = driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[2]/div/div[2]/h2').text
            d[i]['name'] = name
            
            #job title
            try:  
                job = driver.find_element(By.XPATH, '//h4[@class = "style__Job-cmp__sc-1s7e137-2 dzeTcv"]').text
            except NoSuchElementException:
                job = driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[2]/div/div[2]/h4[1]').text
            d[i]['job title'] = job
            
            #organisation
            #some delegates don't have organisation
            #to handle this error I have slighly modified the error
            try:
                org = driver.find_element(By.XPATH, '//h3[@class = "style__Organization-cmp__sc-1s7e137-3 cVzOUy"]').text
            except NoSuchElementException:
                try:
                    org = driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[2]/div/div[2]/h3').text
                except NoSuchElementException:
                    org = ''
            d[i]['organisation'] = org
            
            #question answers  
            qu_ans = driver.find_elements(By.XPATH, '//div[@class = "style__Wrapper-cmp__sc-165xmjy-0 kxlYqL"]')
            for j in range(0, len(qu_ans)):
                q = qu_ans[j].text.split('\n')[0]
                a = qu_ans[j].text.split('\n')[1]
                d[i][q] = a
            
            #check progress
            if i != 0 and i%50 == 0:
                print(f'Extracted {i} delegates')
              
           
        #convert the dictionary to datafranme & transpose
        df = pd.DataFrame.from_dict(d)
        df1 = df.T
        
        #close the driver
        driver.close()
        
        return df1
        
        

        
        
        

# Webscrapping

instantiate the class

In [35]:
dr = DelegateRoster()

get the links

In [12]:
links = dr.get_links()
links

['https://ocimpact.app.swapcard.com/widget/event/oc2023/person/RXZlbnRQZW9wbGVfMjIzMTY3Mjg=',
 'https://ocimpact.app.swapcard.com/widget/event/oc2023/person/RXZlbnRQZW9wbGVfMjEzNzMyNDE=',
 'https://ocimpact.app.swapcard.com/widget/event/oc2023/person/RXZlbnRQZW9wbGVfMjEzNzM0NjE=',
 'https://ocimpact.app.swapcard.com/widget/event/oc2023/person/RXZlbnRQZW9wbGVfMjMyNzcwMjQ=',
 'https://ocimpact.app.swapcard.com/widget/event/oc2023/person/RXZlbnRQZW9wbGVfMjUzNzMxMDE=',
 'https://ocimpact.app.swapcard.com/widget/event/oc2023/person/RXZlbnRQZW9wbGVfMjUyOTQxMzQ=',
 'https://ocimpact.app.swapcard.com/widget/event/oc2023/person/RXZlbnRQZW9wbGVfMjUyODMwNTg=',
 'https://ocimpact.app.swapcard.com/widget/event/oc2023/person/RXZlbnRQZW9wbGVfMjUyODIzMTA=',
 'https://ocimpact.app.swapcard.com/widget/event/oc2023/person/RXZlbnRQZW9wbGVfMjUyNTM3NzQ=',
 'https://ocimpact.app.swapcard.com/widget/event/oc2023/person/RXZlbnRQZW9wbGVfMjUxNTM3OTM=',
 'https://ocimpact.app.swapcard.com/widget/event/oc2023/pers

In [13]:
len(links)

443

There are over **443 delegates**. We will save all the links in **YAML file.**

In [59]:
with open('delegates_links.yaml', 'w') as f:
    yaml.dump(links, f)

Creating dataframe

In [36]:
#extract all info from the links & save it in dataframe
df = dr.create_dataframe(links, limit = len(links))

Extracted 50 delegates
Extracted 100 delegates
Extracted 150 delegates
Extracted 200 delegates
Extracted 250 delegates
Extracted 300 delegates
Extracted 350 delegates
Extracted 400 delegates


In [37]:
df

,name,job title,organisation,What are you working on in the field of social and economic justice?,What do you need from the Delegate community to move this work forward?,What are you able and willing to contribute to the Delegate community?,I'm comfortable conversing in:,Sustainable Development Goals (SDGs),Geographic Focus,Other Geographic Focus Details,Ribbons,Tags
0,Rotimi Olawale (he/him),Executive Director,Youthhubafrica,I am working on providing educational access a...,I'll need technical advise on having further c...,I have extensive experience working in Nigeria...,English,NaN,NaN,NaN,NaN,NaN
1,Nik Kafka (he/him),CEO & Founder,Teach A Man To Fish,Relevant real-world education. Setting young p...,"Partners, funders, external perspectives on ou...",Many years of experience in the for-purpose se...,"English, French, Spanish",NaN,NaN,NaN,NaN,NaN
2,Martin Burt (he/him),Founder & CEO,Fundación Paraguaya & Poverty Stoplight,Two anti-poverty social innovations: the Pover...,Networking,Almost 40 years of experience as a successful ...,NaN,NaN,NaN,NaN,NaN,NaN
3,Maryam Montague (she/her),Founder + Executive Director,Project Soar,I am an Ashoka Arab World Fellow and CNN Afric...,I would like to meet potential donors and part...,"I would be able to contribute my knowledge, pr...","English, French",NaN,NaN,NaN,NaN,NaN
4,Ingrid Karangwayire (she/her),CEO,BK Foundation,"BK Foundation supports programs in Education, ...",BK Foundation would be keen on making partners...,Exploring partnership opportunities.,"English, French",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
438,Alejandra Maria Lorenzana Zuniga (she/her),Executive Director,Sociedad Amigos de los Niños (SAN),NaN,NaN,NaN,"English, Spanish",1. No Poverty3. Good Health and Well-Being4. Q...,Central America,NaN,NaN,NaN
439,Steve Williams,Founding director.,Kyaninga Child Development Centre,I'm working towards helping children living wi...,My goal is to engage and connect with stakehol...,"Thanks to my journey, I've been able to explor...",English,NaN,NaN,NaN,NaN,NaN
440,Abbas Moloo (he/him),Executive Director,Help Aid Africa,"We are working on education scholarships, food...",Fresh ideas and challenges to be prepared for,My experience,"English, Cutchi, Guajarate, Hindi, Swahili, Urdu",NaN,NaN,NaN,NaN,NaN
441,Dunia Colomba (he/him),Founder and Executive Director,VISION GLOBALE D'ORIENTATION DES JEUNES,my organisation works in eduating congolese on...,New connections and expériences through thé ar...,Exchange my experience with others and talk ab...,French,NaN,NaN,NaN,NaN,NaN


In [38]:
len(df)

443

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 443 entries, 0 to 442
Data columns (total 12 columns):
 #   Column                                                                   Non-Null Count  Dtype 
---  ------                                                                   --------------  ----- 
 0   name                                                                     443 non-null    object
 1   job title                                                                443 non-null    object
 2   organisation                                                             443 non-null    object
 3   What are you working on in the field of social and economic justice?     411 non-null    object
 4   What do you need from the Delegate community to move this work forward?  406 non-null    object
 5   What are you able and willing to contribute to the Delegate community?   403 non-null    object
 6   I'm comfortable conversing in:                                           413 non-n

For some delegates, organisations were blank. So I will replace the blank with NaN values.

In [51]:
org_missing = df[df['organisation'] == ''].index
org_missing

Int64Index([426, 427, 428, 429, 430], dtype='int64')

In [52]:
df['organisation'].replace({'': np.nan}, inplace = True)

In [55]:
df.iloc[org_missing]

,name,job title,organisation,What are you working on in the field of social and economic justice?,What do you need from the Delegate community to move this work forward?,What are you able and willing to contribute to the Delegate community?,I'm comfortable conversing in:,Sustainable Development Goals (SDGs),Geographic Focus,Other Geographic Focus Details,Ribbons,Tags
426,Will Meyers (he/him),Software Engineer,NaN,"Software development and data mining, cleaning...",Thought partners on how best to use data minin...,"Technical and data skills, including data mini...",English,4. Quality Education10. Reduced Inequalities13...,NaN,NaN,NaN,NaN
427,Caren McNelly McCormack (she/her),"Capacity Builder, Consultant",NaN,Consulting with organizations to build capacit...,Thoughts and experience for best consulting pr...,Industry experience and a listening ear.,English,1. No Poverty2. Zero Hunger3. Good Health and ...,NaN,NaN,CatalystOC365,NaN
428,Stefanie Weiland (she/her),Social Impact Consultant,NaN,I spent the last 12 years leading global healt...,I need to experience the catalytic conversatio...,I am ready to contribute my energy and encoura...,"English, French",NaN,NaN,NaN,NaN,NaN
429,David Nichols (he/him),Impact Investor,NaN,Invested in and advising climate and renewable...,Connections and networking. Possible co-invest...,"Connections and networking, advisory roles, de...",English,NaN,NaN,NaN,NaN,NaN
430,Karen Shoren Hagel (she/her),"Educator, sustainability in schools consultant...",NaN,I am working with youth with the aim of develo...,Build community,I can present or talk to others about teaching...,English,4. Quality Education,North America,NaN,NaN,NaN


For the above employees organisation is not provided.

In [53]:
df.isnull().sum()

name                                                                         0
job title                                                                    0
organisation                                                                 5
What are you working on in the field of social and economic justice?        32
What do you need from the Delegate community to move this work forward?     37
What are you able and willing to contribute to the Delegate community?      40
I'm comfortable conversing in:                                              30
Sustainable Development Goals (SDGs)                                       307
Geographic Focus                                                           317
Other Geographic Focus Details                                             379
Ribbons                                                                    350
Tags                                                                       416
dtype: int64

A lot of values are missing. I have currently kept as it is. Filling these missing values will require research & domain expertise

Columns like **Sustainable Development Goals (SDGs) , Geographic Focus, Other Geographic Focus Details, Ribbons, Tags** have over 70% missing values. These coluns can be dropped...

Regarding the **name** column there are pronouns as well. We can separate the pronouns using split function & create a separate column for gender.

The information about all 443 delegates has been successfully extracted. Saving it in **CSV.**

In [39]:
df.to_csv('delegates_info.csv', index = False)

# **Connecting to duckdb and importing csv in duckdb**

Temporary databse

In [41]:
duckdb.read_csv('delegates_info.csv', header = True)

┌──────────────────────┬──────────────────────┬──────────────────────┬───┬──────────────────────┬────────────────┐
│         name         │      job title       │     organisation     │ … │       Ribbons        │      Tags      │
│       varchar        │       varchar        │       varchar        │   │       varchar        │    varchar     │
├──────────────────────┼──────────────────────┼──────────────────────┼───┼──────────────────────┼────────────────┤
│ Rotimi Olawale (he…  │ Executive Director   │ Youthhubafrica       │ … │ NULL                 │ NULL           │
│ Nik Kafka (he/him)   │ CEO & Founder        │ Teach A Man To Fish  │ … │ NULL                 │ NULL           │
│ Martin Burt (he/him) │ Founder & CEO        │ Fundación Paraguay…  │ … │ NULL                 │ NULL           │
│ Maryam Montague (s…  │ Founder + Executiv…  │ Project Soar         │ … │ NULL                 │ NULL           │
│ Ingrid Karangwayir…  │ CEO                  │ BK Foundation        │ … │ NULL 

Saving the database and loading..

In [3]:
df = pd.read_csv('delegates_info.csv')
cursor = duckdb.connect(r"delegates_db.db", read_only = False)
cursor.register("delegates_db",df)
cursor.commit()

In [5]:
cursor.execute("SELECT * FROM delegates_db").fetchdf()

,name,job title,organisation,What are you working on in the field of social and economic justice?,What do you need from the Delegate community to move this work forward?,What are you able and willing to contribute to the Delegate community?,I'm comfortable conversing in:,Sustainable Development Goals (SDGs),Geographic Focus,Other Geographic Focus Details,Ribbons,Tags
0,Rotimi Olawale (he/him),Executive Director,Youthhubafrica,I am working on providing educational access a...,I'll need technical advise on having further c...,I have extensive experience working in Nigeria...,English,None,None,None,None,None
1,Nik Kafka (he/him),CEO & Founder,Teach A Man To Fish,Relevant real-world education. Setting young p...,"Partners, funders, external perspectives on ou...",Many years of experience in the for-purpose se...,"English, French, Spanish",None,None,None,None,None
2,Martin Burt (he/him),Founder & CEO,Fundación Paraguaya & Poverty Stoplight,Two anti-poverty social innovations: the Pover...,Networking,Almost 40 years of experience as a successful ...,None,None,None,None,None,None
3,Maryam Montague (she/her),Founder + Executive Director,Project Soar,I am an Ashoka Arab World Fellow and CNN Afric...,I would like to meet potential donors and part...,"I would be able to contribute my knowledge, pr...","English, French",None,None,None,None,None
4,Ingrid Karangwayire (she/her),CEO,BK Foundation,"BK Foundation supports programs in Education, ...",BK Foundation would be keen on making partners...,Exploring partnership opportunities.,"English, French",None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
438,Alejandra Maria Lorenzana Zuniga (she/her),Executive Director,Sociedad Amigos de los Niños (SAN),None,None,None,"English, Spanish",1. No Poverty3. Good Health and Well-Being4. Q...,Central America,None,None,None
439,Steve Williams,Founding director.,Kyaninga Child Development Centre,I'm working towards helping children living wi...,My goal is to engage and connect with stakehol...,"Thanks to my journey, I've been able to explor...",English,None,None,None,None,None
440,Abbas Moloo (he/him),Executive Director,Help Aid Africa,"We are working on education scholarships, food...",Fresh ideas and challenges to be prepared for,My experience,"English, Cutchi, Guajarate, Hindi, Swahili, Urdu",None,None,None,None,None
441,Dunia Colomba (he/him),Founder and Executive Director,VISION GLOBALE D'ORIENTATION DES JEUNES,my organisation works in eduating congolese on...,New connections and expériences through thé ar...,Exchange my experience with others and talk ab...,French,None,None,None,None,None
